# CalgaryHacks2026 IBM Workshop - Intro

Welcome! In this notebook, you will:
- connect to an LLM API
- send a simple prompt
- stream the response live in the notebook


## Before you run

1. Install Python (version 3.12.12 used for development)
2. Install UV and run `uv sync`
3. Generate your Ollama API Key
4. Create a `.env` file in this project folder using `cp .env.example .env`
5. Add your API key to the `.env` file like this:

```env
OLLAMA_API_KEY=your_real_api_key_here
```
6. Select the correct kernel (top-right in Jupyter)

### Optional: Run Granite locally

- Granite models: `https://ollama.com/search?q=granite`
- Local Ollama default host/port: `http://localhost:11434`
- In the client cell below, uncomment the local connection block and comment out the cloud block.


In [19]:
import os
from dotenv import load_dotenv
from ollama import Client
from IPython.display import display, Markdown


In [20]:
# Load values from .env
load_dotenv()

api_key = os.getenv("OLLAMA_API_KEY", "").strip()

if not api_key or api_key == "your_api_key_here":
    raise ValueError(
        "Missing OLLAMA_API_KEY. Add it to your .env file, then rerun this cell."
    )

display(Markdown("API key loaded successfully."))


API key loaded successfully.

In [21]:
# Create API client using Ollama Cloud (default)
client = Client(
    host="https://ollama.com",
    headers={"Authorization": f"Bearer {api_key}"},
)

display(Markdown("Client created."))

Client created.

### Optional: Granite 4 local setup (separate path)

If you want to run locally instead on your own hardware instead of the cloud:

1. Install/start Ollama locally (`http://localhost:11434` is the default host/port).
2. Pull a Granite model from: https://ollama.com/search?q=granite: `ollama pull granite4:latest`
3. Use the optional local client cell below (uncomment).
4. Then either skip the model-check cell, or set `MODEL_NAME = "granite4:latest"` in that cell.


In [22]:
# Optional local client
# Uncomment this block if using a Ollamam locally instead of the cloud API.

# client = Client(
#     host="http://localhost:11434",
# )
# local_model = "granite4:latest"
# display(Markdown(f"Using local Ollama + {local_model}"))

In [23]:
# Connection + model availability check
MODEL_NAME = "qwen3-next:80b"

try:
    model_data = client.list()
except Exception as exc:
    raise ConnectionError(
        "Could not connect to Ollama API. Check internet access and API key."
    ) from exc

available_models = [m.get("model", "") for m in model_data.get("models", [])]

if not available_models:
    raise RuntimeError(
        "Connected, but no models are available for this account."
    )

if MODEL_NAME in available_models:
    selected_model = MODEL_NAME
    display(Markdown(f"Found requested model: `{selected_model}`"))
else:
    selected_model = available_models[0]
    display(Markdown(
        f"`{MODEL_NAME}` not found. Using `{selected_model}` instead."
    ))

n = 10
preview = ", ".join(available_models[:n])
display(Markdown(f"Available models (first n): `{preview}`"))


Found requested model: `qwen3-next:80b`

Available models (first n): `cogito-2.1:671b, glm-4.6, glm-4.7, glm-5, kimi-k2:1t, kimi-k2.5, kimi-k2-thinking, qwen3-coder:480b, qwen3-next:80b, qwen3-coder-next`

In [24]:
user_prompt = "Tell me a one-line fun fact about the University of Calgary"

messages = [
    {
        "role": "user",
        "content": user_prompt,
    },
]

In [25]:
resp = client.chat(model=selected_model, messages=messages, stream=False)
text = resp.get("message", {}).get("content", "")
display(Markdown(text))



The University of Calgary campus sits atop a fossil-rich layer where over 100 dinosaur skeletons have been unearthed—making it one of the most dinosaur-dense university grounds in the world! 🦖

In [26]:
system_prompt = "Welcome to CalgaryHacks 2026! Encourage the user for this 24 hour long hackathon and provide them with great ideas."

user_prompt = "Can you suggest a great hackathon idea to impress the judges?"

messages = [
    {
        "role": "system",
        "content": system_prompt,
    },
    {
        "role": "user",
        "content": user_prompt,
    },
]

In [27]:
resp_2 = client.chat(model=selected_model, messages=messages, stream=False)
text_2 = resp_2.get("message", {}).get("content", "")
display(Markdown(text_2))



Hey there, CalgaryHacks 2026 champ! 👋 First off—**congrats on signing up for this 24-hour whirlwind of creativity and grit!** Whether you're a coding wizard, a design guru, or just someone with big ideas, this is your chance to build something that *matters*. Judges don’t just look for "cool tech"—they want **real-world impact, local relevance, and a story that sticks with them**. Forget perfection; focus on **one clear problem, one innovative solution, and a demo that makes them say, "Wow, I wish I’d thought of that!"**

Here’s a **proven idea tailored for CalgaryHacks 2026** that checks all the boxes:  
🔥 **"EcoPulse: AI-Powered Retrofits for Calgary’s Aging Buildings"** 🔥  

### 🌍 Why This Stands Out  
- **Calgary-Specific Problem**:  
  Calgary has over **100,000 buildings built before 2000**—many are energy hogs. The city’s *Net Zero by 2050* plan needs solutions *now*, but retrofitting is expensive and confusing for homeowners. Existing tools are either too technical (for engineers) or too generic (like "turn off lights").  
- **The Twist**:  
  We’re not just another energy app. **EcoPulse uses AI to turn public data into hyper-local, actionable steps**—like "Your 1970s bungalow in Inglewood could save $300/year by replacing this window *and* adding attic insulation." It’s built for **homeowners, not experts**, with plain English advice.  
- **Why Judges Will Love It**:  
  - ✅ **Ties to Calgary’s future**: Directly supports the city’s climate goals (Calgary Climate Initiative, Alberta’s Net Zero Act).  
  - ✅ **Uses real public data**: Leverages the [City of Calgary’s Open Data Portal](https://data.calgary.ca/) (e.g., building age, energy reports, neighborhood maps).  
  - ✅ **Simple but powerful tech**: A web app with a map interface + AI recommendations (using lightweight ML like TensorFlow.js—no heavy cloud costs!).  
  - ✅ **Scalable impact**: Could partner with Calgary’s "Green Home Program" or Alberta Energy to offer rebates.  

### 💡 How to Build It in 24 Hours (MVP Focus)  
| Component | What to Build | Tools to Use (Free & Easy) |  
|-----------|---------------|----------------------------|  
| **Data Layer** | Pull building age, energy use, and location data from Calgary’s Open Data Portal | Python (Pandas) + City of Calgary API |  
| **AI Engine** | "What retrofits save the most for *your* home?"—use a simple rule-based model (e.g., "If building age > 1980, prioritize windows; if < 1960, prioritize insulation") | TensorFlow.js (no training needed—just logic rules) |  
| **Frontend** | A clean, mobile-friendly map where users click their neighborhood → see "Your Home’s EcoScore" + 3 actionable tips | React + Leaflet.js (for maps) |  
| **Demo** | Show a real example: "This house in Bridgeland has an EcoScore of 42%—here’s how to get it to 70% with $2k in upgrades" | Figma for mockups, then build a live prototype in 6 hours |  

### 🚀 Why This Wins  
- **It’s not just code—it’s a story**: Start your pitch with: *"Imagine a grandmother in a 1950s house in Bowness. She’s worried about high bills and climate change, but doesn’t know where to start. EcoPulse gives her a clear path—no jargon, just savings."*  
- **Real partnerships**: Mention you’d partner with **Calgary Environmental Network** or **Alberta Energy** for credibility (even if just in the pitch).  
- **Scalable vision**: "This could grow into a city-wide retrofit program—imagine Calgary becoming North America’s greenest city by 2030."  

### 💡 Backup Ideas (If You Want Options)  
- **"Wildfire Guardian"**: A voice-first app for Elders in wildfire-prone areas (e.g., Kananaskis) that alerts them to smoke via phone camera + AI. Uses **Google’s Speech-to-Text API** + **TensorFlow Lite** for image analysis. *Why it works*: Solves a *critical* Alberta problem with accessibility at its core.  
- **"RezConnect"**: A voice-based app for Indigenous communities to access healthcare, language learning, or local news—no typing needed. Integrate **Cree/Siksika language support** via open-source tools. *Why it works*: Deeply respectful, socially impactful, and addresses Calgary’s Indigenous population (1 in 8 Calgarians are Indigenous).  

### 🏆 Pro Tips to Impress Judges  
1. **Start with the problem, not the tech**: "Calgary has 100,000 aging buildings wasting energy. Here’s how we fix it."  
2. **Show, don’t tell**: Have a *live demo* of your MVP—even if it’s just 1-2 features working.  
3. **Keep it human**: Show a photo of a real Calgary home, or a quote from a resident.  
4. **Mention partnerships**: "We’d work with the City of Calgary to integrate this into their Green Home Program."  
5. **Own the "why"**: "This isn’t just about saving money—it’s about making Calgary resilient for future generations."  

> 💬 **Remember**: Judges see 100+ projects. The one that **solves a local problem with heart, simplicity, and real data** will stand out. You don’t need a fully polished app—just a **compelling prototype + a story that makes them feel it matters**.  

You’ve got this! 🌟 Calgary’s future is being built right here, right now. Go make something that changes lives. And when you’re coding at 3 AM, remember: **24 hours is enough to change the world.**  

*P.S. Need help brainstorming your pitch? Reply with your idea—I’ll give you feedback!* 💬

## Try this next
- Change the user prompt to your own hackathon theme.
- Ask for a 30-second pitch and a simple MVP plan.
- Ask for judging criteria and how to optimize for each one.
